<a href="https://colab.research.google.com/github/starirene9/Algorithm-Udemy/blob/master/MungMungMusicMaker_K2024504%EA%B5%AC%EB%B9%9B%EB%82%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1단계: demucs 설치
!pip install -U demucs

# 2단계: 필요한 라이브러리 설치
!pip install -U gradio torch torchaudio gTTS

In [ ]:
from demucs.apply import apply_model
from demucs.pretrained import get_model
import gradio as gr
import os
import torchaudio
import torch
from gtts import gTTS

# Demucs 모델 불러오기 (1번만 실행)
model = get_model('htdemucs')  # 최신 고성능 모델

# 텍스트를 음성(wav)으로 저장하는 함수
def tts_to_wav(text, filename, lang='ko'):
    tts = gTTS(text=text, lang=lang)
    mp3_path = f"{filename}.mp3"
    wav_path = f"{filename}.wav"
    tts.save(mp3_path)
    waveform, sr = torchaudio.load(mp3_path)
    torchaudio.save(wav_path, waveform, sr)
    return wav_path
"""
	•	waveform: 오디오 진폭 배열 (행렬 형태)
	•	sr: 샘플레이트 (예: 22050Hz)를 반환
"""

def mix_sounds(base_wave, insert_wave, insert_position, sr, mix_volume=0.5):
    insert_sample = int(insert_position * sr)
    insert_wave = insert_wave * mix_volume
    T = base_wave.shape[1]
    t = insert_wave.shape[1]

    # 필요 시 base_wave 뒤에 zero padding 추가
    if insert_sample + t > T:
        pad_length = insert_sample + t - T
        pad = torch.zeros((2, pad_length))
        base_wave = torch.cat([base_wave, pad], dim=1)

    # mix
    base_wave[:, insert_sample:insert_sample + t] += insert_wave
    return base_wave
"""
# mix_sounds 함수 설명
# base_wave: 배경음 오디오 (Tensor: [채널, 길이])
# insert_wave: 삽입할 효과음 오디오 (Tensor: [채널, 길이])
# insert_position: 몇 초 지점에 삽입할지 (예: 3.5초)
# sr: 샘플레이트 (예: 44100Hz)
# mix_volume: 효과음 볼륨 (0.0 ~ 1.0 사이)
"""

# 오디오 분리 + 효과음 추가 함수
def separate_audio(input_audio, extra_sound=None, add_walk=False, add_snack=False, custom_tts=None):
    basename = os.path.basename(input_audio).replace('.mp3', '').replace('.wav', '')
    os.makedirs("output", exist_ok=True)

    # 입력 오디오 로드 및 10초로 자르기
    audio_wave, sr = torchaudio.load(input_audio)
    if audio_wave.shape[0] == 1:
        audio_wave = audio_wave.expand(2, -1)
    max_length = sr * 10
    if audio_wave.shape[1] > max_length:
        audio_wave = audio_wave[:, :max_length]

    # Demucs로 보컬 분리
    sources = apply_model(model, audio_wave.unsqueeze(0), split=True)
    vocals = sources[0][model.sources.index("vocals")]
    accompaniment = sources[0][model.sources.index("other")]

    add_sounds = []

    # 산책 효과음
    if add_walk:
        walk_path = tts_to_wav("산책 갈까?", "output/walk")
        walk_wave, _ = torchaudio.load(walk_path)
        if walk_wave.shape[0] == 1:
            walk_wave = walk_wave.expand(2, -1)
        add_sounds.append(walk_wave)

    # 간식 효과음
    if add_snack:
        snack_path = tts_to_wav("간식 먹을래?", "output/snack")
        snack_wave, _ = torchaudio.load(snack_path)
        if snack_wave.shape[0] == 1:
            snack_wave = snack_wave.expand(2, -1)
        add_sounds.append(snack_wave)

    # 사용자 입력 TTS
    if custom_tts:
        custom_path = tts_to_wav(custom_tts, "output/custom")
        custom_wave, _ = torchaudio.load(custom_path)
        if custom_wave.shape[0] == 1:
            custom_wave = custom_wave.expand(2, -1)
        add_sounds.append(custom_wave)

    # 추가 사운드 처리 (강아지 짖는 소리)
    if extra_sound:
        sound1, sr1 = torchaudio.load(extra_sound)
        if sound1.shape[0] == 1:
            sound1 = sound1.expand(2, -1)
        if sound1.shape[1] > sr1 * 10:
            sound1 = sound1[:, :sr1 * 10]
        add_sounds.append(sound1)

    insert_point = 4.0
    for s in add_sounds:
        accompaniment = mix_sounds(accompaniment, s, insert_point, sr, mix_volume=0.5)
        insert_point += 2.0  # 다음 효과음은 그보다 뒤에 삽입

    vocal_path = f"output/{basename}_vocals.wav"
    bg_path = f"output/{basename}_bg.wav"

    torchaudio.save(vocal_path, vocals, sr)
    torchaudio.save(bg_path, accompaniment, sr)

    return (
        gr.update(value=vocal_path, visible=True),
        gr.update(value=bg_path, visible=True)
    )
"""
sound.shape[0] → 채널 수 (1: mono, 2: stereo)
sound.shape[1] → 전체 샘플 수 (시간 길이)

if sound.shape[0] == 1:
    sound = sound.expand(2, -1)
	•	mono 오디오는 한쪽밖에 없어서 stereo로 만들기 위해 복사해서 두 채널로 확장하는 거야!
	•	-1은 기존 길이를 유지하라는 뜻이야
"""

# Gradio UI (초기화 버튼 포함)
with gr.Blocks() as demo:
    gr.Markdown("## 🐶 멍멍뮤직메이커 🎵 세상에 하나뿐인, 우리 강아지만을 위한 맞춤 음악!")
    gr.Markdown("""
      이 앱은 당신이 선택한 음악에서 **목소리와 배경음을 자동으로 분리**하고,
      **“산책 갈까?”, “간식 먹을까?” 같은 TTS 메시지나 강아지 짖는 소리**를
      내가 좋아하는 배경 음악에 **귀엽게 믹스**해줍니다.

      🐾 산책할 때, 🍖 간식 줄 때!
      우리 강아지를 위한 **특별한 플레이리스트**, 지금 바로 만들어보세요! 💖
      """)

    with gr.Row():
        input_audio = gr.Audio(type="filepath", label="🎵 입력 오디오 (mp3/wav)")
        extra_audio = gr.Audio(type="filepath", label="🎤 추가 강아지 사운드 (선택, 10초 이내)")

    with gr.Column():
        add_walk = gr.Checkbox(label="🐾 산책 갈까? 효과음")
        add_snack = gr.Checkbox(label="🍖 간식 먹을래? 효과음")
        custom_tts = gr.Textbox(label="🗣 사용자 지정 TTS 메시지 (선택)", placeholder="예) 사랑해, 우리 강아지~")

    with gr.Row():
        submit_btn = gr.Button("🎬 변환 시작")
        clear_btn = gr.Button("🔄 입력 초기화")

    with gr.Row():
        output_vocals = gr.Audio(label="🗣 목소리만", visible=False)
        output_bg = gr.Audio(label="🎼 배경음 + 효과음", visible=False)

    submit_btn.click(
        fn=separate_audio,
        inputs=[input_audio, extra_audio, add_walk, add_snack, custom_tts],
        outputs=[output_vocals, output_bg]
    )

    clear_btn.click(
        fn=lambda: [None, None, False, False, None, None],
        inputs=[],
        outputs=[input_audio, extra_audio, add_walk, add_snack, output_vocals, output_bg]
    )

# 앱 실행
if __name__ == "__main__":
    demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4a647b6175afafe1ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
